In [ ]:
# Source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import os
import pandas as pd

In [2]:
os.chdir(r'C:\Users\lenovo\Desktop')

In [13]:
reviews_df=pd.read_csv('final_reviews.csv')

In [14]:
reviews_df

,rating,title,work/life,culture/values,career_opp,compensation/ben,senior_mng,pros,cons
0,5,Great place to work,5,5,3,4,4,"Employee friendly, Great Culture, Strong Values","None, really a great place to work"
1,5,awesome work life balance,5,5,5,5,5,work life balance is great,none ..its a very good place
2,5,Employee Centric,5,5,5,5,4,Employee friendly .Think about their growth,you will fall in love with the compnay
3,4,"Great Company to work for, one of the best in ...",4,4,4,4,4,"Good work environment , very good work culture",Less acknowledgment and less challenging work
4,5,Nice company to work at all hierarchy,5,5,4,3,4,1. opportunities to grow are ample 2. Great wo...,"1. Salaries lesser than MS, AMZ, etc."
5,4,Nice Company,4,4,5,4,3,High salary Limited work hours nice culture,High competition Skills Demanded Hard Work
6,5,Good place to Work - Great policies &amp; bene...,5,5,4,5,4,Great Policies Great work culture Employee Fri...,Shift timings are the only cons.
7,3,"Good Company, Bad Division (PPBU)",2,5,2,4,1,"Amazing benefits, Good food and facilities",Internal politics. Bad work life balance since...
8,1,Adobe India Document Cloud Server Team,1,5,1,4,1,Overall company benefits and facilities.,"Very rude management, managers, no considerati..."
9,5,Computer Scientist at Adobe,5,5,5,5,5,Work culture is very good,Adobe doesn't provide pickup and drop service


In [15]:
reviews_df.drop(['rating','work/life','culture/values','career_opp','compensation/ben','senior_mng'], inplace=True,  axis=1)

In [16]:
def corpus(row):
    try:
        title=' '.join(list(row[0].split())[:10])
    except:
        title=''
    try:
        pros=' '.join(list(row[1].split())[:40])
    except:
        pros=''
    try:
        cons=' '.join(list(row[2].split())[:50])
    except:
        cons=''
    
    return title+' '+pros+' '+cons
reviews_df['corpus']=reviews_df.apply(corpus,axis=1)

In [99]:
from tqdm import tqdm
tqdm.pandas()
import nltk
sentence_re = r'(?:(?:[A-Z])(?:.[A-Z])+.?)|(?:\w+(?:-\w+)*)|(?:\$?\d+(?:.\d+)?%?)|(?:...|)(?:[][.,;"\'?():-_`])'
lemmatizer = nltk.WordNetLemmatizer()
grammar = r"""
        NBAR:
            {<NN.?|JJ>?<NN.?>*}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """

def fff(text):

    chunker = nltk.RegexpParser(grammar)   
    toks = nltk.regexp_tokenize(text, sentence_re)
    postoks = nltk.tag.pos_tag(toks)
    tree = chunker.parse(postoks)
    from nltk.corpus import stopwords
    stopwords = stopwords.words('english')

    def leaves(tree):
        """Finds NP (nounphrase) leaf nodes of a chunk tree."""
        for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()

    def normalise(word):
        """Normalises words to lowercase and stems and lemmatizes it."""
        word = word.lower()
        word = lemmatizer.lemmatize(word)
        return word

    def acceptable_word(word):
        """Checks conditions for acceptable word: length, stopword. We can increase the length if we want to consider large phrase"""
        accepted = bool(2 <= len(word) <= 40
            and word.lower() not in stopwords)
        return accepted


    def get_terms(tree):
        for leaf in leaves(tree):
            term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
            term=' '.join(term)
            yield term

    terms = list(get_terms(tree))
    return(terms)
    
reviews_df['corpus_nouns']=reviews_df['corpus'].progress_apply(fff)

100%|██████████████████████████████████████████████████████████████████████████| 165252/165252 [55:14<00:00, 49.86it/s]


In [21]:
import re
import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\lenovo\Anaconda3new\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [102]:
corpus_list=list(reviews_df['corpus_nouns'])

In [110]:
corpus_list

[['great place',
  'employee',
  'great culture',
  'strong value none',
  'great place'],
 ['awesome work life balance work life balance', 'great none', 'good place'],
 ['employee centric employee', 'growth', 'love', 'compnay'],
 ['great company',
  'good work environment',
  'good work culture le acknowledgment',
  'challenging work'],
 ['nice company',
  'opportunity',
  'ample',
  'great work environment',
  'employee experience satisfaction',
  'salary',
  ' amz'],
 ['nice company high salary limited work hour',
  'nice culture high competition skill demanded hard work'],
 ['good place',
  ' - g reat policy',
  'benefit great',
  'great work culture employee friendly lot',
  'additional benefit',
  'wellness',
  'education',
  'meal',
  'shift timing',
  'con'],
 ['good company',
  'bad division  (pp bu',
  'amazing benefit',
  'good food',
  'facility internal politics',
  'bad work life balance',
  'sale people',
  'night shift',
  'focus',
  'product',
  'photoshop',
  'acrobat

In [112]:
corpus_list[0]

['great place',
 'employee',
 'great culture',
 'strong value none',
 'great place']

In [113]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(corpus_list, min_count=3, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus_list], threshold=10)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [120]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'great'])

In [138]:
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [139]:
corpus_list_nostops = remove_stopwords(corpus_list)

# Form Bigrams
corpus_list_bigrams = make_bigrams(corpus_list_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = en_core_web_sm.load(disable=['parser', 'ner'])

[['great', 'place', 'employee', 'great', 'culture', 'strong', 'value', 'none', 'great', 'place']]


In [168]:
data_lemmatized=corpus_list_bigrams

In [169]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [174]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100000,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)

In [176]:
pprint(lda_model.print_topics())

[(0,
  '0.017*"year" + 0.016*"good place" + 0.011*"work" + 0.011*"con" + '
  '0.007*"good company" + 0.007*"opportunity" + 0.007*"good work environment" '
  '+ 0.006*"good work life balance" + 0.006*"good" + 0.005*"great place"'),
 (1,
  '0.049*"company" + 0.017*"people" + 0.016*"good company" + 0.013*"microsoft" '
  '+ 0.013*"lot" + 0.009*"employee" + 0.008*"opportunity" + 0.007*"great '
  'company" + 0.007*"manager" + 0.006*"career"'),
 (2,
  '0.012*"company" + 0.008*"flexible" + 0.007*"people" + 0.006*"lot" + '
  '0.005*"opportunity" + 0.004*"culture" + 0.004*"great company" + 0.004*"good '
  'place" + 0.004*"infosys" + 0.003*"management"'),
 (3,
  '0.077*"good" + 0.018*"employee" + 0.016*"fresher" + 0.015*"company" + '
  '0.014*"good company" + 0.014*"lot" + 0.012*"work" + 0.011*"project" + '
  '0.010*"work life balance" + 0.009*"salary"'),
 (4,
  '0.024*"great place" + 0.011*"oracle" + 0.010*"growth" + 0.008*"con" + '
  '0.008*"work" + 0.007*"place" + 0.006*"salary" + 0.006*"oppor

In [177]:
doc_lda = lda_model[corpus]

In [179]:
lda_model.save('lda.model')

In [180]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\lenovo\Anaconda3new\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.112128 -0.101818       1        1  27.576912
7      0.106508  0.053629       2        1  23.279427
1      0.111457  0.063805       3        1  17.918934
0     -0.043393 -0.099758       4        1   7.151997
6     -0.047759  0.041578       5        1   6.808367
4     -0.077202  0.016669       6        1   6.034214
5     -0.077039 -0.035180       7        1   5.967268
2     -0.084700  0.061075       8        1   5.262876, topic_info=      Category          Freq                    Term         Total  loglift  \
term                                                                          
95     Default  29320.000000                    good  29320.000000  30.0000   
115    Default  21810.000000                 company  21810.000000  29.0000   
2      Default   6169.000000             great place   6169.000000  28.0000   
24668  Default   3321.000000                  oracle   3321.000000  27.0000   
169    Default   6629.000000                    year   6629.000000  26.0000   
1878   Default   4746.000000               microsoft   4746.000000  25.0000   
5      Default   6656.000000              good place   6656.000000  24.0000   
39     Default   9523.000000            good company   9523.000000  23.0000   
482    Default   6392.000000                 fresher   6392.000000  22.0000   
141    Default  19728.000000                     lot  19728.000000  21.0000   
27     Default   4643.000000                     con   4643.000000  20.0000   
114    Default  10667.000000                    work  10667.000000  19.0000   
186    Default  11232.000000                  people  11232.000000  18.0000   
127    Default   5530.000000                 benefit   5530.000000  17.0000   
9      Default   3434.000000                  growth   3434.000000  16.0000   
14     Default   4135.000000           great company   4135.000000  15.0000   
53     Default   8510.000000                 project   8510.000000  14.0000   
131    Default   4831.000000                   place   4831.000000  13.0000   
630    Default   1607.000000                flexible   1607.000000  12.0000   
21     Default   6887.000000                  salary   6887.000000  11.0000   
389    Default   8162.000000                    time   8162.000000  10.0000   
0      Default  11350.000000                employee  11350.000000   9.0000   
1042   Default   3339.000000                    hike   3339.000000   8.0000   
13     Default    771.000000   good work environment    771.000000   7.0000   
1393   Default    994.000000              nice place    994.000000   6.0000   
205    Default   5375.000000       work life balance   5375.000000   5.0000   
202    Default   2112.000000  good work life balance   2112.000000   4.0000   
43     Default   2167.000000                 product   2167.000000   3.0000   
5353   Default    990.000000                exposure    990.000000   2.0000   
1461   Default   1909.000000                    hour   1909.000000   1.0000   
...        ...           ...                     ...           ...      ...   
162     Topic8    183.168610             good people   1422.982178   0.8944   
651     Topic8    148.823837                   india   1047.469482   0.9931   
644     Topic8    276.988800                 culture   3426.111816   0.4293   
22942   Topic8     52.255199          indian company    156.518997   1.8475   
186     Topic8    478.689026                  people  11232.785156  -0.2110   
7283    Topic8     29.279030           many politics     55.414249   2.3065   
14      Topic8    258.551178           great company   4135.255371   0.1723   
1036    Topic8    143.824692       good work culture   1298.997803   0.7437   
2691    Topic8     74.010002                  stress    346.480225   1.4009   
282     Topic8    180.016098            good benefit   2167.934570   0.4560   
20      Topic8    34